# Comprehensive test suite for the InfoBuddy.AI Server

This notebook provides a comprehensive implementation of a chat assistant using:
- <xyz> llm for conversational AI
- Tavily Search for real-time web information
- LangGraph for workflow orchestration
- Memory management for conversation continuity

Features:
- Class-based architecture with clear separation of concerns
- Comprehensive error handling and logging
- Type safety with complete annotations
- Debugging and visualization capabilities
- Industry best practices implementation.

## dependencies

In [2]:
from typing import List, Optional, TypedDict, Any, Union, Annotated, Dict
from langgraph.graph import add_messages, StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4

import os
os.makedirs("logs", exist_ok=True)
import sys
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s | %(levelname)-8s | %(name)s | %(message)s",
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(filename='logs/test.log', mode='a')
    ]
)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
load_dotenv()

True

## schema / state definitions

In [3]:
class ChatConversationState(TypedDict):
    """
    Enhanced state definition for chat conversation management.
    
    This TypedDict defines the structure of the conversation state that flows
    through the LangGraph workflow, ensuring type safety and clear data contracts.
    
    Attributes:
        messages: List of conversation messages with automatic message addition functionality
    """
    messages: Annotated[List[BaseMessage], add_messages]

## chat assistant model

In [9]:
class ChatAssistantNotebook:
    """
    Production-ready chat assistant implementation for Jupyter notebooks.
    
    This class encapsulates all chat assistant functionality including:
    - LLM integration with <xyz> llm
    - Web search capabilities via Tavily
    - Conversation memory management
    - Graph-based workflow orchestration
    - Comprehensive error handling and logging
    
    Attributes:
        model: <xyz> model instance
        search_tool: Tavily search tool for web queries
        memory_manager: Conversation memory checkpoint manager
        conversation_graph: Compiled LangGraph workflow
        
    Example:
        >>> assistant = ChatAssistantNotebook()
        >>> await assistant.process_user_message("What's the weather today?")
    """
    def __init__(
            self,
            model_name: str = 'gemini-2.5-flash',
            max_search_results: int = 2,
    ):
        logger.info("🚀 Initializing Enhanced Chat Assistant...")
        
        self.model_name = model_name
        self.max_search_results = max_search_results
        self.app = None
        self.model = None
        self.search_tool = None
        self.memory = None
        self.available_tools = []
        self.llm = None
        self.enhanced_llm = None

        try:
            
            self._initialize_language_model()
            self._initialize_search_capabilities()
            self._initialize_memory_management()
            self._setup_conversation_workflow()
            logger.info("✅ Chat Assistant initialized successfully.")
        except Exception as e:
            logger.error(f"❌ Initialization failed: {e}")
            raise

    def _initialize_language_model(self) -> None:
        """ Initialize the <xyz> language model"""
        try:
            self.llm = ChatGoogleGenerativeAI(model=self.model_name, temperature=0.1)
            logger.info(f"✅ Language model '{self.model_name}' initialized.")
        except Exception as e:
            logger.error(f"❌ Failed to initialize language model: {e}")
            raise

    def _initialize_search_capabilities(self) -> None:
        """ Initialize the Tavily search tool """
        try:
            self.search_tool = TavilySearchResults(max_results=self.max_search_results)
            self.available_tools = [self.search_tool]

            # bind tools to llm
            self.enhanced_llm = self.llm.bind_tools(tools=self.available_tools)

            logger.info("✅ Search tool initialized and enhances llm capabilities.")
        except Exception as e:
            logger.error(f"❌ Failed to initialize search tool capabilities: {e}")
            raise

    def _initialize_memory_management(self) -> None:
        """ Initialize the conversation memory manager"""
        try: 
            self.memory = MemorySaver()
            logger.info("✅ Memory manager initialized.")
        except Exception as e:
            logger.error(f"❌ Failed to initialize memory manager: {e}")
            raise

    def _setup_conversation_workflow(self) -> None:
        """ Setup the LangGraph conversation workflow"""
        try:
            # create the state graph
            graph = StateGraph(ChatConversationState)

            # add nodes
            graph.add_node('llm_node', self._execute_language_model_processing)
            graph.add_node('tool_execution_node', self._execute_tool_operations)

            # set entry point
            graph.set_entry_point('llm_node')

            # add edges
            graph.add_conditional_edges(
                'llm_node', 
                self._route_to_appropriate_node,
                {
                    'tool_execution_node': 'tool_execution_node',
                    END: END
                }
            )
            graph.add_edge('tool_execution_node', 'llm_node')

            # compile the graph
            self.app = graph.compile(checkpointer=self.memory)

            logger.info("✅ Conversation workflow graph created and compiled.")
        except Exception as e:
            logger.error(f"❌ Failed to create state graph: {e}")
            raise

    async def _execute_language_model_processing(self, state: ChatConversationState) -> Dict[str, List[BaseMessage]]:
        """ Execute the language model processing step"""
        try:
            logger.debug("🧠 Processing messages through language model...")
            res = await self.enhanced_llm.ainvoke(state['messages'])
            logger.debug(f"🔍 Model response: {res}")
            return {'messages': res}
        except Exception as e:
            logger.error(f"❌ LLM processing failed: {e}")
            raise

    async def _route_to_appropriate_node(self, state: ChatConversationState) -> str:
        """ Determine the next node in the workflow"""
        try:
            last_msg = state['messages'][-1]

            # check if last message contains a tool call
            has_tool_calls = (
                hasattr(last_msg, 'tool_calls') and
                len(last_msg.tool_calls) > 0
            )
            if has_tool_calls:
                logger.info(f"🔧 Tool execution required: {len(last_msg.tool_calls)} calls")
                return 'tool_execution_node'
            else:
                logger.debug("✅ No tool calls detected, ending workflow.")
                return END
        except Exception as e:
            logger.error(f"❌ Node routing failed: {e}")
            raise

    async def _execute_tool_operations(self, state: ChatConversationState) -> Dict[str, List[ToolMessage]]:
        """ Execute any required tool operations"""
        try:
            logger.info("🔧 Executing tool operations...")
            tool_calls = state['messages'][-1].tool_calls
            executed_tools = []

            for t in tool_calls:
                logger.debug(f"🔍 Executing tool: {t['name']} with input: {t['args']}")
                tool_result = await self._process_individual_tool_call(t)
                executed_tools.append(tool_result)
            
            logger.debug(f"✅ Executed {len(executed_tools)} & its results: {executed_tools}")
            return {'messages': executed_tools}
        except Exception as e:
            logger.error(f"❌ Tool execution failed: {e}")
            return {'messages': []}
        
    async def _process_individual_tool_call(self, tool_call: Dict[str, Any]) -> ToolMessage:
        """ Process an individual tool call"""
        tool_name = tool_call['name']
        tool_args = tool_call.get('args', {})
        tool_identifier = tool_call['id']

        logger.debug(f"🔍 Processing tool call id: {tool_identifier} || name: {tool_name} || with args: {tool_args}")
        try:
            if tool_name == self.search_tool.name:
                search_results = await self.search_tool.ainvoke(tool_args)
                tool_msg = ToolMessage(
                    content=str(search_results),
                    tool_call_id=tool_identifier,
                    name=tool_name
                )
                logger.info(f"🔍 Search completed for query: {tool_args.get('query', 'unknown')}")
                return tool_msg 
            else:
                logger.warning(f"⚠️ Unknown tool requested: {tool_name}")
                return ToolMessage(
                    content=f"Unknown tool: {tool_name}",
                    tool_call_id=tool_identifier,
                    name=tool_name
                )
        except Exception as e:
            logger.error(f"❌ Tool {tool_name} execution failed: {str(e)}")
            return ToolMessage(
                content=f"Tool execution failed: {str(e)}",
                tool_call_id=tool_identifier,
                name=tool_name
            )
        
    def visualize_conversation_workflow(self) -> None:
        """ Visualize the conversation workflow graph"""
        try:
            graph_obj_getter = getattr(self.app, 'get_graph', None)
            graph_obj = graph_obj_getter() if callable(graph_obj_getter) else self._graph

            # 2. Mermaid source ---------------------------------------------------
            if hasattr(graph_obj, 'draw_mermaid'):
                try:
                    mermaid_src = graph_obj.draw_mermaid()
                    print(mermaid_src)
                    logger.info("🧪 Mermaid diagram (text) printed.")
                except Exception as e:
                    logger.debug("⚠️ Mermaid render not available: %s", e)

            # 3. ASCII fallback ---------------------------------------------------
            if hasattr(graph_obj, 'draw_ascii'):
                try:
                    ascii_map = graph_obj.draw_ascii()
                    print(ascii_map)
                    logger.info("📄 ASCII graph printed.")
                    return
                except Exception as e:
                    logger.debug("⚠️ ASCII render not available: %s", e)
        except Exception as e:
            logger.warning(f"❌ Graph visualization failed: {e}")

    async def process_user_message(
            self,
            user_message: str,
            conversation_id: Optional[Union[str, int]] = None
    ) -> None:
        """
        Process a user message through the complete conversation workflow.
        
        Args:
            user_message: The message from the user to process
            conversation_id: Optional conversation identifier for memory continuity
        """
        if not user_message or not user_message.strip():
            logger.error("⚠️ Empty user message provided, skipping processing.")
            raise ValueError("User message cannot be empty.")
        
        # generate a conversation id if not provided
        if conversation_id is None:
            conversation_id = str(uuid4())
            logger.debug(f"🆕 Generated new conversation ID: {conversation_id}")
        
        conversation_config = {
            'configurable': {
                'thread_id': conversation_id
            }
        }

        print(f"\n👤 User: {user_message}")
        logger.info(f"👤 User (id: {conversation_id}): {user_message}")
        print("🤖 Assistant: ", end="", flush=True)

        assistant_response = ""
        search_performed = False

        try:
            async for e in self.app.astream(
                {'messages': [HumanMessage(content=user_message)]},
                config=conversation_config,
                version='v2'
            ):
                event_type = e.get('event', '')
                if event_type == 'on_chat_model_stream':
                    chunk = e.get('data', {}).get('chunk')
                    if chunk and hasattr(chunk, 'content'):
                        chunk_content = getattr(chunk, 'content', '')
                        if chunk_content:
                            print(chunk.content, end="", flush=True)
                            logger.debug(f"💬 Streaming: {chunk.content}")
                            assistant_response += chunk.content
                elif event_type == 'on_tool_start':
                    tool_name = e.get('data', {}).get('tool_name', '')
                    if tool_name == "tavily_search_results_json":
                        if not search_performed:
                            print("\n🔍 Searching the web...", end="", flush=True)
                            logger.debug(f"🔧 Tool starting: {tool_name}")
                            search_performed = True
                elif event_type == "on_tool_end":
                    tool_name = e.get("name", "")
                    if tool_name == "tavily_search_results_json":
                        print("\n🤖 Assistant: ", end="", flush=True)
                        logger.debug(f"✅ Tool completed: {tool_name}")
                print()  
        except Exception as e:
            logger.error(f"❌ Message processing failed: {e}")
            print(f"\n❌ An error occurred: {e}")
            raise

## Run demonstration

In [ ]:
async def demonstrate_chat_assistant():
    """
    Demonstration function showing how to use the enhanced chat assistant.
    
    This function provides examples of:
    - Basic initialization
    - Interactive chat display
    - Message processing with real-time streaming
    - Error handling
    """
    try:
        print("🎯 Starting Chat Assistant Demonstration...")
        print("=" * 60)
        
        # Initialize the chat assistant
        chat_assistant = ChatAssistantNotebook(
            model_name="gemini-2.5-flash",
            max_search_results=2,
        )

        chat_assistant.visualize_conversation_workflow()
        
        print("✅ Chat Assistant initialized successfully!")
        print("=" * 60)
        
        # Process example messages
        test_messages = [
            "Hi, my name is Jiten",
            "When is the next SpaceX launch?",
            "Whats my name?"
        ]
        
        for message in test_messages:
            await chat_assistant.process_user_message(message)
            print("-" * 60)
        
        print("✅ Demonstration completed successfully")
        
    except Exception as demo_error:
        print(f"❌ Demonstration failed: {str(demo_error)}")
        logger.error(f"❌ Demonstration failed: {str(demo_error)}")
        raise

In [11]:
await demonstrate_chat_assistant()

🎯 Starting Chat Assistant Demonstration...
2025-09-06 15:22:40 | INFO     | __main__ | 🚀 Initializing Enhanced Chat Assistant...
2025-09-06 15:22:40 | INFO     | __main__ | ✅ Language model 'gemini-2.5-flash' initialized.
2025-09-06 15:22:40 | INFO     | __main__ | ✅ Search tool initialized and enhances llm capabilities.
2025-09-06 15:22:40 | INFO     | __main__ | ✅ Memory manager initialized.
2025-09-06 15:22:40 | INFO     | __main__ | ✅ Conversation workflow graph created and compiled.
2025-09-06 15:22:40 | INFO     | __main__ | ✅ Chat Assistant initialized successfully.
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	llm_node(llm_node)
	tool_execution_node(tool_execution_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> llm_node;
	llm_node -.-> __end__;
	llm_node -.-> tool_execution_node;
	tool_execution_node --> llm_node;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

2

## Run interactive chat

In [16]:
async def interactive_chat():
    """
    Interactive chat function for manual testing with user input.
    """
    try:
        print("🚀 Starting Interactive Chat Assistant...")
        print("=" * 60)
        
        # Initialize the chat assistant
        chat_assistant = ChatAssistantNotebook(
            model_name="gemini-2.5-flash",
            max_search_results=4,
        )
        
        print("✅ Chat Assistant ready! Type 'quit' to exit.")
        print("=" * 60)
        
        conversation_id = uuid4()
        
        while True:
            try:
                user_input = input("\n👤 You: ").strip()
                
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    print("👋 Goodbye!")
                    break
                
                if not user_input:
                    print("⚠️ Please enter a message.")
                    continue
                
                await chat_assistant.process_user_message(user_input, conversation_id)
                print("-" * 60)
                
            except KeyboardInterrupt:
                print("\n👋 Chat interrupted. Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {str(e)}")
                
    except Exception as init_error:
        print(f"❌ Failed to initialize chat: {str(init_error)}")

In [17]:
await interactive_chat()

🚀 Starting Interactive Chat Assistant...
2025-09-06 14:24:52 | INFO     | __main__ | 🚀 Initializing Enhanced Chat Assistant...
2025-09-06 14:24:52 | INFO     | __main__ | ✅ Language model 'gemini-2.5-flash' initialized.
2025-09-06 14:24:52 | INFO     | __main__ | ✅ Search tool initialized and enhances llm capabilities.
2025-09-06 14:24:52 | INFO     | __main__ | ✅ Memory manager initialized.
2025-09-06 14:24:52 | INFO     | __main__ | ✅ Conversation workflow graph created and compiled.
2025-09-06 14:24:52 | INFO     | __main__ | ✅ Chat Assistant initialized successfully.
✅ Chat Assistant ready! Type 'quit' to exit.

👤 User: hi my name is jiten
2025-09-06 14:25:00 | INFO     | __main__ | 👤 User (id: 285829db-1aaf-4470-85f6-902094fcfa66): hi my name is jiten
🤖 Assistant: 2025-09-06 14:25:00 | DEBUG    | __main__ | 🧠 Processing messages through language model...
2025-09-06 14:25:02 | DEBUG    | __main__ | 🔍 Model response: content='Hi Jiten, nice to meet you! How can I help you today?' ad